In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from pylab import *
import numpy as np

In [2]:
import sys
sys.path.append("../../src/data")
from replication.PMotion import Polymer
from replication.simulate import load_parameters
from replication.tools import load_lengths_and_centro, load_ori_position
from replication.ensembleSim import ensembleSim

HOOMD-blue v2.1.2 CUDA (7.5) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 
Compiled: 01/11/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----


In [30]:
Nsim = 1 
coarse=5000
lengths, _ = load_lengths_and_centro("../../data/external/saccharomyces_cerevisiae_R64-2-1_20150113.gff",
                                     coarse,verbose=False)

  
l_ori = load_ori_position("../../data/external/oridb.csv",
                                  ["Confirmed"],
                                  lengths,
                                  coarse,verbose=False)
p_on=0.01
p_off=0.01
p_v=0.0001
only_one= True
fork_speed = 0.2
dt_speed = 0.1
Ndiff=100
E = ensembleSim(Nsim, l_ori, Ndiff, lengths,
                        p_on=p_on, p_off=p_off, only_one=only_one, 
                     all_same_ori=True,fork_speed=fork_speed,
                    gindin = False,
                    p_v = p_v,
                   dt_speed =dt_speed)

In [31]:
S = E.run_all(2000)


100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


In [32]:
S.record_diffusing[2].build_time_line()

array([ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
        1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
        1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,
        0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0

In [1]:
from IPython.html.widgets import interact, fixed,interact_manual,IntSlider,VBox,jslink
#from ipywidgets import StaticInteract, RangeWidget, RadioWidget
import numpy as np
import ipyvolume
import ipyvolume.pylab as p3
import numpy as np

f = p3.figure()
f.animation = 100

#x,y,z are two dimentionnal array with the first dimension being time
x = np.array([ 2 * t + np.arange(10) for t in range(5)])
y = x**2
z = 3 * x

p3.xlim(0, np.max(x))
p3.ylim(0, np.max(y))
p3.zlim(0, np.max(z))

time = 0
s = p3.scatter(x[time],y[time],z[time], s=0.05,color="rgb(1,0,200)")

def toi(time): 
    s.x = x[time]
    s.y = y[time]
    s.z = z[time]
    
time= IntSlider(min=0,max=4,step=1,value=0)
it = interact(toi,time=time)

vbox = VBox([p3.gcc(),time])


/home/jarbona/miniconda3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [191]:
it.kwargs

AttributeError: 'function' object has no attribute 'kwargs'

In [3]:
import mdtraj as md
sys.path.append("../../src/data")

data_folder = "../../data/cluster/yeast-5kb-wt-nuc-wt-spb-no-half/traj1/"

t = md.load(os.path.join(data_folder, 'poly.dcd'),
            top=os.path.join(data_folder, "atoms.hoomdxml"))

with open(os.path.join(data_folder,"polymer_timing.dat"),"rb") as f:
        lPolymers = cPickle.load(f)


P = [np.array(lPolymers[p].get_replication_profile()) for p in range(len(lPolymers))]


In [13]:
from IPython.html.widgets import interact, fixed,interact_manual,IntSlider,VBox,jslink
#from ipywidgets import StaticInteract, RangeWidget, RadioWidget
import numpy as np
import ipyvolume
import ipyvolume.pylab as p3
import numpy as np

from colour import Color


import numpy as np
try:
    import _pickle as cPickle
except:
    import cPickle
import copy
import sys
import os

sys.path.append("../")

# Create a figure
f = p3.figure(width=1200,height=1200)
f.animation = 0
# Tell visual to use this as the viewer.

# A silly visualization.
color_d = {}
color_d["green"] = (0, 128, 0)
# Even sillier animation.



class Representation:
    def __init__(self,traj,what,by_resid=True,tube=False,t=0,update=False,time_color=None,color="resid"):
        self.first = True
        self.traj = traj
        self.selections = []
        self.time_color = time_color
        self.tube = tube
        self.color = color
        self.colors = []
        self.named_selections = []

        n_chains = len(list(self.traj.topology.chains))

        if color == "resid":
            self.colors = []

        if by_resid:
            for i in range(n_chains):
                add = ""
                if what != "":
                    add = " and %s" % what
                sele = "resid %i" % i + add

                self.named_selections.append(sele)
                sel = self.traj.topology.select(sele)

                # print(sele,len(sel))
                if len(sel) != 0:
                    self.selections.append(sel)
                    
            
            if color == "resid":
                red = Color("red")
                blue = Color("blue")
                self.colors = [c.hex for c in red.range_to(blue, len(self.selections))]
                
        else:
            self.named_selections.append(what)
            sel = self.traj.topology.select(what)
            if len(sel) != 0:
                self.selections.append(sel)
            else:
                print("Empty selection", what)

    def draw(self, time):
        # print(self.selections)
        if self.first:

            self.rep = []

        for isel, sel in enumerate(self.selections):
            x, y, z = self.traj.xyz[time, sel].T
            print(np.min(x),np.max(x))
            if self.time_color is not None:


                colors = self.time_color(time)
                if len(colors) < isel + 1:
                    print("The colouring function did not cover all the selection")
                    print(isel, self.named_selections[isel])
                    print(len(x), isel)
                    raise

                colors = colors[isel]

                if len(x) != len(colors):
                    print(len(x),len(colors),isel)
                    print(self.named_selections[isel])
                    raise
            else:
                colors = [1 for i in range(len(x))]

            #print(len(x),len(colors))

            if self.first:
                if self.tube:
                    vx = np.concatenate( (x[1:] - x[:-1],[0]))
                    vy = np.concatenate( (y[1:] - y[:-1],[0]))
                    vz = np.concatenate( (z[1:] - z[:-1],[0]))
                    self.rep.append(p3.quiver(x, y, z,vx,vy,vz,s=0.05,color=self.color))
                else:
                    #print(self.color)
                    color = self.color
                    if self.color == "resid":
                        color = self.colors[isel]
                    
                    self.rep.append(p3.scatter(x, y, z,s=0.0003,color=color,marker="sphere"))
            else:

                self.rep[isel].x = x
                self.rep[isel].y = y
                self.rep[isel].z = z
                #,y=y,z=z,scalars=colors)#,scalars=replic)
        self.first = False
#print(x.shape,y.shape)




def time_color(time):
    replic = copy.deepcopy(P)
    for l in range(len(P)):
        replic[l][replic[l] > time / 10. ] = 0
        replic[l][replic[l] != 0 ] = 2
        replic[l][replic[l] == 0 ] = 1
    #print(len(replic))
    return replic

Reprs = [#Representation(traj=t,what="name Diff",tube=False,by_resid=False,color="green"),
         #Representation(traj=t,what="name Telo",tube=False,by_resid=False,color="green"),
         #Representation(traj=t,what=" (not name Diff) and (not name Spb) and resid 1",tube=True,color="red"),
         Representation(traj=t,what=" (not name Diff) and (not name Spb) and resid 3",tube=False,color="red")]

        # Representation(traj=t,what=" (not name Diff) and (not name Spb)",tube=True,time_color=time_color)]
for r in Reprs:
    r.draw(0)

R=1.
p3.xlim(-R, R)
p3.ylim(-R, R)
p3.zlim(-R, R)
#Load the replication information:
def toi(stime,btime):
    for r in Reprs:
        r.draw(stime+10*btime)


#time = IntSlider(min=0, max=100, step=1)

#jslink((Reprs[0].rep[0], 'index'), (time, 'value'))

vbox = VBox([p3.gcc(),time])
vbox

-0.839632 1.22289


In [17]:
Reprs[0].rep[0].size=0.001

In [172]:
import time as Time
f.animation=10
for time in range(100):
    for r in Reprs:
        #print(time)
        Time.sleep(0.1)
        r.draw(time)

In [174]:
ipyvolume.embed.embed_html("test.html",vbox)